In [1]:
import os
import sys
module_path = os.path.join(os.path.abspath(os.path.join('..')))
if module_path not in sys.path:
    sys.path.append(module_path)

In [15]:
def flatten(S):
    if S == []:
        return S

    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])

    return S[:1] + flatten(S[1:])

def iterate_and_write_file(data_generator, filename: str):
    path = os.path.join(module_path, 'var', filename)

    # Clear file
    with open(path, 'w') as f:
        f.write('')

    f = open(path, "a")

    for i, (params, overlap) in enumerate(data_generator):
        params = flatten(params)
        line = ','.join([str(p) for p in params]) + ',' + str(overlap if isinstance(overlap, int) else ','.join([str(o) for o in overlap]))

        f.write(line + "\n")

    f.close()

## 2 components

In [16]:
from moebius.data.generator import NumericGridRange, GridIterator, EnsureSumGridIterator, GaussianMixtureDataGenerator

w = EnsureSumGridIterator(
    GridIterator([
        NumericGridRange(0.1, 1.0, 0.1), NumericGridRange(0.1, 1.0, 0.1)
    ]),
    1
)

means = GridIterator([
    GridIterator([ NumericGridRange.unit(0, 3), NumericGridRange.unit(0, 3)]),
    GridIterator([ NumericGridRange.unit(3, 5), NumericGridRange.unit(3, 5)])
])


covs = GridIterator([
    GridIterator([
        GridIterator([ NumericGridRange.unit(1, 3), NumericGridRange.const(0)]),
        GridIterator([ NumericGridRange.const(0), NumericGridRange.unit(1, 3)])
    ]),
    GridIterator([
        GridIterator([ NumericGridRange.const(1.52), NumericGridRange.const(1.1)]),
        GridIterator([ NumericGridRange.const(1.1), NumericGridRange.const(1.52)])
    ])
])

iterator = GridIterator([w, means, covs])
data_generator = GaussianMixtureDataGenerator(iterator)

iterate_and_write_file(data_generator, 'mixtures.csv')

## 3 components

In [18]:
w = EnsureSumGridIterator(
    GridIterator([
        NumericGridRange(0.1, 1.0, 0.1), NumericGridRange(0.1, 1.0, 0.1), NumericGridRange(0.1, 1.0, 0.1)
    ]),
    1
)

means = GridIterator([
    GridIterator([ NumericGridRange.unit(0, 3), NumericGridRange.unit(0, 3)]),
    GridIterator([ NumericGridRange.unit(3, 5), NumericGridRange.unit(3, 5)]),
    GridIterator([ NumericGridRange.unit(3, 5), NumericGridRange.unit(3, 5)]),
])


covs = GridIterator([
    GridIterator([
        GridIterator([ NumericGridRange.unit(1, 3), NumericGridRange.const(0)]),
        GridIterator([ NumericGridRange.const(0), NumericGridRange.unit(1, 3)])
    ]),
    GridIterator([
        GridIterator([ NumericGridRange.const(1.52), NumericGridRange.const(1.1)]),
        GridIterator([ NumericGridRange.const(1.1), NumericGridRange.const(1.52)])
    ]),
    GridIterator([
        GridIterator([ NumericGridRange.const(1.52), NumericGridRange.const(1.1)]),
        GridIterator([ NumericGridRange.const(1.1), NumericGridRange.const(1.52)])
    ])
])

iterator = GridIterator([w, means, covs])
data_generator = GaussianMixtureDataGenerator(iterator)

iterate_and_write_file(data_generator, 'mixtures_3.csv')

In [ ]:
from multiprocessing import Process
from typing import Callable

def wrap(i, call):
    i_local = i
    
    return lambda : call(i_local)

def run_tasks_in_parallel(tasks: list[Callable]):
    running_tasks = [Process(target=wrap(i, task)) for i, task in enumerate(tasks)]
    for running_task in running_tasks:
        running_task.start()
    for running_task in running_tasks:
        running_task.join()

def create_tasks_to_compute_olr(iterator: GridIterator, call: Callable, threads: int):
    splitted = iterator.split(threads)
    tasks = []
    
    for s in splitted:
        data_generator = GaussuanMixtureDataGenerator(iterator)
        tasks.append(call)
    
    return tasks

In [10]:
def flatten(S):
    if S == []:
        return S

    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])

    return S[:1] + flatten(S[1:])

def t(thread: int):
    path = f"/opt/notebooks/data/{thread}.csv"
    # Clear file
    with open(path, 'w') as f:
        f.write('')

    f = open(path, "a")
    result = []

    i = 0
    for (params, overlap) in data_generator:
        print_in_out(params, overlap)

        result.append([params, overlap])

        if i != 0 and i % 50 == 0:
            write_to_file(f, result)

            f.flush()

            result = []

        i += 1

    write_to_file(f, result)
    f.close()

def print_in_out(params, overlap):
    print(str(params) + '  ->  ' + str(overlap))

def write_to_file(f, result):
    for params, overlap in result:
        params = flattern(params)
        line = ','.join([str(p) for p in params]) + ',' + str(overlap[0])

        f.write(line + "\n")

    pass

run_tasks_in_parallel(create_tasks_to_compute_olr(iterator, t, 3))